In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import math
import time, datetime
#띄어쓰기
from pycrfsuite_spacing import TemplateGenerator
from pycrfsuite_spacing import CharacterFeatureTransformer
from pycrfsuite_spacing import PyCRFSuiteSpacing
from soyspacing.countbase import CountSpace

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score #정확도 계산

from pprint import pprint
 
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
 
# spacy for lemmatization
import spacy
 
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim # don't skip this
 
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
 
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.min_rows', 500)
pd.set_option('display.max_colwidth', -1)


# 데이터 체크

In [4]:
train = pd.read_csv("C:/cook/dacon/train.csv")
test = pd.read_csv("C:/cook/dacon/public_test.csv")

In [5]:
train.columns
test.columns

Index(['id', 'year_month', 'text', 'smishing'], dtype='object')

Index(['id', 'year_month', 'text'], dtype='object')

In [6]:
train.index
test.index

RangeIndex(start=0, stop=295945, step=1)

RangeIndex(start=0, stop=1626, step=1)

# 분석 시작

In [7]:
train['year_month'].min(), train['year_month'].max(),test['year_month'].min(), test['year_month'].max()

('2017-01', '2018-12', '2019-01', '2019-04')

In [8]:
date = []
p = re.compile('2017-')
q = re.compile('2018-')
for i in train['year_month']:
    if p.match(i):
        date.append(int(i.replace("2017-", "")))
    elif q.match(i):
        date.append(int(i.replace("2018-", "")) + 12)
    else:
        date.append(int(i.replace("2019-", "")) + 24)

In [9]:
season = list(range(0,len(date)))
for i in range(0,len(date)):
    season[i] = math.ceil(date[i] / 3)
train["season"] = season

In [10]:
date = []
p = re.compile('2017-')
q = re.compile('2018-')
for i in test['year_month']:
    if p.match(i):
        date.append(int(i.replace("2017-", "")))
    elif q.match(i):
        date.append(int(i.replace("2018-", "")) + 12)
    else:
        date.append(int(i.replace("2019-", "")) + 24)

In [11]:
season = list(range(0,len(date)))
for i in range(0,len(date)):
    season[i] = math.ceil(date[i] / 3)
test["season"] = season

# 계절 9값 확인 -- 단순 예측(12월 값 *3)
len(train[idx][train['season']==9]), 1150


len(train[idy][train['season']==9])) 4131

# 텍스트 데이터 분석

idx = 스미싱 문자
idy = 정상 문자

head를 보면 정상 문자의 경우, 단순고객접촉을 의도로 발송하는 경우가 보임.
그러나, 스미싱의 경우 이익을 목적으로 하지 않는 문자는 발송하지 않음.
따라서, Document의 주제를 분석하여 비이익접촉을 걸러내는 방법 활용

In [12]:
idx = train['smishing'] == 1
idy = train['smishing'] == 0

데이터에 "ID:" 라는 텍스트가 있으면 모두 스미싱
 -- 카카오톡 플러스친구(공식계정)가 아닌 친구(비공식계정) 등록을 유도하여 속이는 방법으로 추정

# 텍스트 길이 분석

In [13]:
length_x = [] # 스미싱 문자
length_y = [] # 정상 문자

for i in train[idx]['text']:
    length_x.append(len(i))    
for i in train[idy]['text']:
    length_y.append(len(i))

In [14]:
length_x = pd.DataFrame(length_x)
length_y = pd.DataFrame(length_y)
length_x.describe()
length_y.describe()

,0
count,18703.000000
mean,801.004117
std,206.203451
min,39.000000
25%,689.000000
50%,876.000000
75%,917.000000
max,1230.000000


,0
count,277242.000000
mean,133.742395
std,149.335550
min,1.000000
25%,39.000000
50%,75.000000
75%,172.000000
max,1498.000000


길이가 300 ~ 350(whisker 교차지점, 스미싱 whisker 시작부분), 900 ~ 950(스미싱 중앙값 부근)에서 기울기가 급격히 변함

In [6]:
length_text= []
for i in train.values:
    length_text.append([i[0],len(i[2])])
length_text = pd.DataFrame(length_text)

In [7]:
length_text.rename(columns={0:"id", 1:"length_text"}, inplace =True)
length_text

,id,length_text
0,0,24
1,1,37
2,2,81
3,4,174
4,5,40
...,...,...
295940,336373,168
295941,336375,33
295942,336376,97
295943,336377,66


# 띄어쓰기와 형태소 분석

# crf spacing

In [17]:
begin_time = datetime.datetime.now()
print(begin_time)
to_feature = CharacterFeatureTransformer(
    TemplateGenerator()
    )

model_path = 'demo_model.crfsuite'
correct = PyCRFSuiteSpacing(to_feature)
correct.load_tagger(model_path)
end_time = datetime.datetime.now()
print(end_time - begin_time)

2020-01-06 10:37:31.032834
0:00:01.100667


# soy spacing

In [74]:
train['clean_doc'] = train['text']

# 삭제용
time = ['\d+년[가-힣]{0,1}',
        '\d+월[가-힣]{0,1}',
        '\d+일[가-힣]{0,1}',
        '[(]월[)]',
        '[(]화[)]',
        '[(]수[)]',
        '[(]목[)]',
        '[(]금[)]',
        '[(]토[)]',
        '[(]일[)]']

money = ["\d+억{0,1}만{0,1}원", "\d*일*\d*십*\d*백*\d*천*\d*만*\d*억*\d*원"]


# 변수 추가용
anboo = ['감사','행복', '사랑', '즐거', '고맙']
myeonjeol = ['새해', '추석', '설날', '구정', '신정', '한가위', '명절', '연휴']
sorry = ['파업', '불편', '양해', '사과', '죄송', '이해', '이전', '이동', '발령']
callme = ['문자', '연락', '전화', '답장', '상담']
visit = ['내점', '방문']
daechool = ['대출', '대환', '증액', '채무', '연체', '원리금', '부채', '상환', '']
event = ["이벤트", "당첨", "추첨", "응모"]
interest = ['\d[.]\d+\%']
link = ['http']
kakao = ['톡\s*상담', 'KISA', 'ID\s*:']

def make_col(target, colname_str):
    colname = colname_str
    train[colname] = 0
    
    for t in target:
        count = -1
        p = re.compile(t)
        
        for i in train['clean_doc']:
            count += 1
            
            if p.findall(i):
                train[colname][count] += 1

def del_text(target):
    train['clean_doc'] = train['clean_doc'].str.replace(target," ")

In [19]:
make_col(anboo, "anboo")
make_col(myeonjeol, "myeongjeol")
make_col(sorry, "sorry")
make_col(callme, "callme")
make_col(visit, "visit")
make_col(daechool, "daechool")
make_col(event, "event")
make_col(interest, "interest")
make_col(link, "link")
make_col(kakao, "kakao")

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
train['clean_doc'] = train['text']

for m in money:
    del_text(m)

for i in time:
    del_text(i)

train['clean_doc'] = train['clean_doc'].str.replace("[^가-힣+]"," ")

In [182]:
begin_time = datetime.datetime.now()
print(begin_time)
clean_doc2 = []
for i in train['clean_doc']:
    clean_doc2.append(correct(i))

end_time = datetime.datetime.now()
print(end_time- begin_time)

2020-01-06 12:05:11.752601
0:10:04.405361


In [172]:
train.iloc[181303]

id            207367                                
year_month    2017-11                               
text          11월 내내 건강과 행복이 함께하세요. XXX은행 XXX점 XXX올림
smishing      0                                     
season        4                                     
clean_doc       월 내내 건강과 행복이 함께하세요     은행    점    올림
anboo         1                                     
myeongjeol    0                                     
sorry         0                                     
callme        0                                     
visit         0                                     
daechool      0                                     
event         0                                     
interest      0                                     
link          0                                     
kakao         0                                     
detoken       월 내내 건강 과 행복 이 함께 하 세요 은행 점 올림        
Name: 181303, dtype: object

In [183]:
train['clean_doc'] = clean_doc2

In [184]:
train[train['clean_doc'].isna()==True]

,id,year_month,text,smishing,season,clean_doc,anboo,myeongjeol,sorry,callme,visit,daechool,event,interest,link,kakao,detoken


In [185]:
train['clean_doc'].fillna(" ", inplace = True)

In [186]:
train.to_csv("train_prep.csv", index=False)

# 주제 찾기 시작

In [187]:
from eunjeon import Mecab
tagger = Mecab()

In [188]:
result = []
for i in train['clean_doc']:
    result.append(tagger.morphs(i))

In [189]:
detokenized_doc = []
for i in range(len(result)):
    t = ' '.join(result[i])
    detokenized_doc.append(t)

detokenized_doc[0]

'은행 성산 팀장 입니다 행복 한 주말 되 세요'

In [190]:
train['detoken'] = detokenized_doc

In [192]:
train[train['detoken'].isna() == True]

,id,year_month,text,smishing,season,clean_doc,anboo,myeongjeol,sorry,callme,visit,daechool,event,interest,link,kakao,detoken


In [9]:
train = pd.read_csv("train_detoken.csv")
train = pd.merge(train, length_text, how = 'inner')
train

,id,year_month,text,smishing,season,clean_doc,anboo,myeongjeol,sorry,callme,visit,daechool,event,interest,link,kakao,detoken,length_text
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,1,은행 성산 팀장 입니다 행복한 주말 되세요,1,0,0,0,0,0,0,0,0,0,은행 성산 팀장 입니다 행복 한 주말 되 세요,24
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,1,오늘도 많이 웃으시는 하루 시작하세요 은행 진월동 라운지 올림,0,0,0,0,0,0,0,0,0,0,오늘 도 많이 웃 으시 는 하루 시작 하 세요 은행 진월동 라운지 올림,37
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0,1,안녕하십니까 고객님 은행 입니다 금일 납부하셔야 할 금액은 입니다 감사합니다 새해 ...,1,1,0,0,0,0,0,0,0,0,안녕 하 십니까 고객 님 은행 입니다 금일 납부 하 셔야 할 금액 은 입니다 감사 ...,81
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0,1,고객 님안녕하세요 은행 지점 입니다 지난 한 해 동안 저희 지점에 보내주신 성 에 ...,3,1,0,0,0,0,0,0,0,0,고객 님 안녕 하 세요 은행 지점 입니다 지난 한 해 동안 저희 지점 에 보내 주 ...,174
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,1,새로 움이 가득 입니다 올 한해 더 많이 행복한 한해 되시길 바랍니다,1,0,0,0,0,0,0,0,0,0,새로 움 이 가득 입니다 올 한 해 더 많이 행복 한 한 해 되 시 길 바랍니다,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295940,336373,2018-12,XXX 고객님!열심히 달려왔던 2018년도 어느 새 뒤안길로 지나쳐가고 벅찬 설렘으...,0,8,고객님 열심히 달려왔던 어느 새 뒤안길로 지나쳐 가고 벅찬 설렘으로 신년의 새로운 ...,2,1,0,0,0,0,0,0,0,0,고객 님 열심히 달려왔 던 어느 새 뒤안길 로 지나쳐 가 고 벅찬 설렘 으로 신년 ...,168
295941,336375,2018-12,XXX고객님실버웰빙신탁이 만기도래 예정입니다.XXX남양주지점,0,8,고객 님실버 웰빙 신탁이 만기도 래 예정입니다 남양주지점,0,0,0,0,0,0,0,0,0,0,고객 님 실버 웰빙 신탁 이 만 기 도 래 예정 입니다 남양주 지점,33
295942,336376,2018-12,한해동안 XXX은행과 함께 해주셔서 정말 감사드립니다 2019년 기해년을 맞이하며 ...,0,8,한해 동안 은행과 함께 해주셔서 정말 감사드립니다 기해 년을 맞이하며 가족의 건강과...,2,1,0,0,0,0,0,0,0,0,한 해 동안 은행 과 함께 해 주 셔서 정말 감사 드립니다 기해 년 을 맞이 하 며...,97
295943,336377,2018-12,1228(금)예금및 적금 만기입니다.예금은 시간내서 내점하시고 적금은 1년 자동연장...,0,8,예금및 적금 만기입니다 예금은 시간 내서 내점하시고 적금은 자동연장되니 참고바랍니다...,0,0,0,0,1,0,0,0,0,0,예금 및 적금 만기 입니다 예금 은 시간 내서 내점 하 시 고 적금 은 자동 연장 ...,66


In [10]:
train.to_csv("train_detoken.csv", index = False)

In [136]:
train_set = train.drop(['year_month','text','id', 'clean_doc'], axis=1)
train_set

,smishing,season,anboo,myeongjeol,sorry,callme,visit,daechool,event,interest,link,kakao,detoken
0,0,1,1,0,0,0,0,0,0,0,0,0,은행 성산 팀장 입니다 행복 한 주말 되 세요
1,0,1,0,0,0,0,0,0,0,0,0,0,오늘 도 많이 웃 으시 는 하루 시작 하 세요 은행 진월동 라운지 올림
2,0,1,1,1,0,0,0,0,0,0,0,0,안녕 하 십니까 고객 님 은행 입니다 금일 납부 하 셔야 할 금액 은 원 입니다 감사 합니다 새해 복 많이 받 으십시오 은행 옥포 올림
3,0,1,3,1,0,0,0,0,0,0,0,0,고객 님 안녕 하 세요 은행 지점 입니다 지난 한 해 동안 저희 지점 에 보내 주 신 성원 에 감사 드립니다 설렘 으로 시작 한 년 소망 하 시 는 일 모두 이 고객 님 의 가정 에 늘 건강 과 행복 이 함께 하 길 기원 하 겠 습니다 사랑 하 는 가족 과 함께 정 을 나누 는 행복 한 설 명절 보내 세요 은행 지점 직원 일동
4,0,1,1,0,0,0,0,0,0,0,0,0,월 은 새로 움 이 가득 입니다 올 한 해 더 많이 행복 한 한 해 되 시 길 바랍니다
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295940,0,8,2,1,0,0,0,0,0,0,0,0,고객 님 열심히 달려왔 던 년 도 어느 새 뒤안길 로 지나쳐 가 고 벅찬 설렘 으로 신년 의 새로운 기운 을 맞이 하 는 시점 입니다 년 에 도 고객 님 과 가정 에 건강 과 행복 이 가득 하 시 기 를 바라 며 원 하 시 는 일 을 이루 시 는 한 해 가 되 셨 으면 합니다 새해 복 많이 받 으십시오 감사 합니다 은행 일 올림
295941,0,8,0,0,0,0,0,0,0,0,0,0,고객 님 실버 웰빙 신탁 이 만 기 도 래 예정 입니다 남양주 지점
295942,0,8,2,1,0,0,0,0,0,0,0,0,한 해 동안 은행 과 함께 해 주 셔서 정말 감사 드립니다 년 기해 년 을 맞이 하 며 가족 의 건강 과 행복 을 기원 합니다 새해 복 많이 받 으십시오 은행 점 올림
295943,0,8,0,0,0,0,1,0,0,0,0,0,금 예금 및 적금 만기 입니다 예금 은 시간 내서 내점 하 시 고 적금 은 년 자동 연장 되 니 참 고 바랍니다 은행 탄현 지점


In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
 
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
 
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]
 
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):   
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
    coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [ ]:
data_words = list(sent_to_words(train['clean_doc']))

print(data_words[:1])

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
 
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
 
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
 
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
 
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])
 
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
 
print(data_lemmatized[:1])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
 
# Create Corpus
texts = data_lemmatized
 
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
 
# View
print(corpus[:1])

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [1]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = 'C:/mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, 
                                             corpus=corpus, 
                                             num_topics=2, 
                                             id2word=id2word)

NameError: name 'gensim' is not defined

In [ ]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))
 
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) # a measure of how good the model is. lower the better.
 
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, 
                                                        corpus=corpus, texts=data_lemmatized, 
                                                        start=2, limit=40, step=6)

In [ ]:
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[!!!!!]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for , row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
    # Get the Dominant topic, Perc Contribution and Keywords for each document
    for j, (topic_num, prop_topic) in enumerate(row):
        if j == 0: # => dominant topic
            wp = ldamodel.show_topic(topic_num)
            topic_keywords = ", ".join([word for word, prop in wp])
            sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
        else:
            break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)
 
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)
 
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
 
# Show
df_dominant_topic.head(10)

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()
 
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')
 
for i, grp in sent_topics_outdf_grpd:
sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet,
grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)],
axis=0)
 
# Reset Index
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)
 
# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]
 
# Show
sent_topics_sorteddf_mallet.head()


In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics